# Installing and Importing necessary libraries

In [ ]:
# installing the required libraries
# !pip install imutils
# !conda install -c conda-forge dlib # If this takes a lot of time, try and run it on anaconda prompt.

In [ ]:
# Importing the libraries
import dlib
import cv2
import smtplib
from scipy.spatial import distance as dist
import numpy as np
import time
import winsound
import imutils
from imutils import face_utils
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
# Build and Install OpenCV With CUDA GPU Support on Windows 10. Reference the link below
# https://www.youtube.com/watch?v=YsmhKar8oOc&t=0s

# To check if the opencv is cuda enabled
print(cv2.__version__)
def is_cuda_cv(): # 1 == using cuda, 0 = not using cuda
    try:
        count = cv2.cuda.getCudaEnabledDeviceCount()
        if count > 0:
            return 1
        else:
            return 0
    except:
        return 0

# Reference:    
# https://stackoverflow.com/questions/61492452/how-to-check-if-opencv-is-using-gpu-or-not

In [ ]:
is_cuda_cv()

# YOLOv3 For Cell phone Detection

In [ ]:
# Loading the YOLO model
net = cv2.dnn.readNet('models/yolov3.weights', 'models/yolov3.cfg')

In [ ]:
#  set CUDA as the preferable backend and target
# To use Graphic card
# This block of code runs only if OpenCV with CUDA is installed.
if is_cuda_cv():
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [ ]:
# Getting all the objects from coco

classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

classes # These are all the objects that YOLOv3 model can detect.

In [ ]:
# Function of object detection using YOLOv3

def yolo_obj_dec(img, net, size, confidence_threshold, threshold, objects):

    # getting the height and width of the image.
    img_height, img_width, _ = img.shape
    
    # Setting up the output layer
    output_layer_names = net.getUnconnectedOutLayersNames()
    # print(output_layer_names)
    layers = net.getLayerNames()
    layers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # print(layerOutputs)
    
    
    # Setting input layer by normalizing the images (Resizing the Image)
    net.setInput(cv2.dnn.blobFromImage(img, 1/255.0, (size, size), swapRB=True, crop=False))
    layerOutputs = net.forward(layers)

    # Bounding boxes, confidences, object_ids
    boxes = []
    confidences = []
    class_ids = []
    
    # getting the info about the condidences,class ids
    # first values are the locations of the bounding boxes
    # fifth value is the box confidence.
    # x,y,wh,w boundary box
    # x,y are center of the object and w and h are size i.e width and the height of the object.
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:] #all the egithy objects predictions
            # print(scores)
            class_id = np.argmax(scores) # high scores 
            confidence = scores[class_id] # high scores
            if confidence > confidence_threshold:
                center_x = int(detection[0]*img_width) # Nomalizing
                center_y = int(detection[1]*img_height)
                w = int(detection[2]*img_width)
                h = int(detection[3]*img_height)
                x = int(center_x - w/2) # for corners
                y = int(center_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                # print(boxes)
                # print(confidences)
    
    # Non maximum suppression to keep the highest scored boxes out of multiple boxes on a same object.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, threshold)
    # print(indexes.flatten()) # To see Redundant boxes
    
    result = []
    if len(indexes) > 0:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            
            result.append((label, confidence, x, y, w, h))
    


    return img_width, img_height,result
    
    
    
# Reference:
# https://www.youtube.com/watch?v=1LCb1PVqzeY

# Drowsiness detection Using Face Landmarks

In [ ]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

# Reference: https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/

In [ ]:
# Initializing the face landmark detectors and predictors

face_landmarks =  'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(face_landmarks)

In [ ]:
# Getting the Co-ordinates of Left-Eye and Right-Eye

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"] # starting and ending cordinates of left eye
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"] # starting and ending cordinates of right eye

# Yawn Detection and Counter Using Face Landmarks

In [ ]:
def Mouth_open(shape):
    upper_lip = np.concatenate((shape[50:53], shape[61:64]))
    low_lip = np.concatenate((shape[56:59], shape[65:68]))

    upper_mean = np.mean( upper_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = abs(upper_mean[1] - low_mean[1])
    return distance

# Reference:
# https://medium.com/analytics-vidhya/yawn-detection-using-opencv-and-dlib-e04ba79b9936

# Alarm Function

In [ ]:
def Alarm():
    winsound.PlaySound('alarm.wav', winsound.SND_FILENAME)

# Mail Function

In [ ]:
def Send_email(txt):
    sender_email_id = 'driveralertAI@gmail.com'
    sender_email_pass = 'Driver@1234'
    receiver_email_id = 'n.abhinay00@gmail.com'
    
    # creates SMTP session 
    s = smtplib.SMTP('smtp.gmail.com', 587)  
    s.starttls() 
    s.login(sender_email_id, sender_email_pass) 
    time = datetime.now()
    curr_time = time.strftime("%H:%M:%S")
    msg = MIMEMultipart()
    msg["From"] = sender_email_id
    msg["To"] = receiver_email_id
    msg["Subject"] = 'Driving Alert triggered at: ' + str(curr_time)
    msg.attach(MIMEText(txt, 'plain'))
    s.sendmail(sender_email_id, receiver_email_id, msg.as_string())

In [ ]:
# Setting thresholds and initializing the counters
Eye_Threshold = 0.27
Eye_AR_Frames = 10 
Drowsy_Counter = 0
Yawn_Threshold = 18
Yawn_Counter = 0
CellPhone_Counter=0
CellPhone_Threshold=10
count_yawn = 0
total_yawn = 0

# Video Capturing and Object detection

In [ ]:
print("Starting the Camera\n")
cap = cv2.VideoCapture(1) # pass 0 if you want to use inbuilt camera, 1 if you want to use USB camera.
while True:
    _, img = cap.read()
    size = 416
    confidence_threshold = 0.5 # Minimum confidence
    threshold = 0.4
    img_width, img_height, result= yolo_obj_dec(img, net, size, confidence_threshold, threshold, classes)

    # Cell Phone Detection
    
    font = cv2.FONT_HERSHEY_PLAIN
    for i in result:
        label, confidence, x, y, w, h = i
        
        if label == "person":
            # print("Person Detected\n")
            cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 255), 2)
            cv2.putText(img, label + confidence + " confidence", (x, y+20), font, 2, (255, 255, 255), 2)
    
        # checking for cell phone in Image
        if label == "cell phone":
            print("cell phone detected\n")
            cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 255), 2)
            cv2.putText(img, label + " Detected "+confidence + " confidence", (x, y+20), font, 2, (255, 255, 255), 2)
            CellPhone_Counter+=1
        
        if CellPhone_Counter >= CellPhone_Threshold:
            Alarm()
            Send_email('Cell Phone usage detected while driving')
            CellPhone_Counter = 0
            
    # Drowsiness Detection & Yawn Detection and Count  
    img = imutils.resize(img, width=450) # 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # converting image to gray scale
    # detecting the face in the image
    rects = detector(gray, 0)
    
    for i in rects:
        shape = predictor(gray, i)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        avg_ear = (leftEAR + rightEAR) / 2.0 #average eye aspect ratio
        # Yawn
        distance = Mouth_open(shape)
        
        # drawing eye boundaries on image
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)

        cv2.drawContours(img, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(img, [rightEyeHull], -1, (0, 255, 0), 1)
        
        # Checking for drowsiness
        if avg_ear < Eye_Threshold:
            Drowsy_Counter += 1
            
            if Drowsy_Counter >= Eye_AR_Frames:
                print('Driver is Feeling Drowsy\n')
                Alarm()
                Send_email('Driver is Feeling Drowsy')
                Drowsy_Counter = 0
                cv2.putText(img, "Drowsiness Detected", (10, 30), font, 2, (255, 255, 255), 2)
            
       # Yawn Detection and Counting    
        else:
            Drowsy_Counter = 0
            if (distance > Yawn_Threshold):
                    Yawn_Counter +=1
                    cv2.putText(img, str(Yawn_Counter), (100, 60),font, 1.5, (0, 0, 255), 2)
                    if Yawn_Counter >= 7:
                        count_yawn += 1
                        total_yawn += 1
                        # print(" Driver is Yawning.")
                        cv2.putText(img, "Yawning", (100, 60),font, 1.5, (0, 0, 255), 2)
                        Yawn_Counter = 0
                        
                    # if greaterthan or equal to three yawns are detected
                    if count_yawn >= 3: 
                        print(" Driver is Yawning. Might be Exhausted. Yawn Count: "+ str(total_yawn))
                        Alarm()
                        Send_email("Driver is Yawning. Might be Exhausted")
                        count_yawn=0
                        
    cv2.putText(img, "EAR: {:.2f}".format(avg_ear), (300, 30), font, 1, (0, 0, 255), 2)
    cv2.putText(img, "Lip Distance: {:.2f}".format(distance), (300, 60), font, 1, (0, 0, 255), 2)
    cv2.putText(img, "Yawn count :{:.2f}".format(total_yawn), (280,90),font, 1, (0, 0, 255), 2)
            
                       
    cv2.imshow('Image', img)
    key = cv2.waitKey(1)
    
    if key == 27: # break if pressed esc key
        break
        
cap.release()

cv2.destroyAllWindows()